In [5]:
import os
# make sure pyspark tells workers to use python3 not 2 if both are installed
#os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3'

import pyspark
conf = pyspark.SparkConf()

# point to mesos master or zookeeper entry (e.g., zk://10.10.10.10:2181/mesos)
conf.setMaster("spark://spark-master:7077")
# point to spark binary package in HDFS or on local filesystem on all slave
# nodes (e.g., file:///opt/spark/spark-2.2.0-bin-hadoop2.7.tgz)
#conf.set("spark.executor.uri", "hdfs://10.122.193.209/spark/spark-2.2.0-bin-hadoop2.7.tgz")
# set other options as desired
#conf.set("spark.executor.memory", "2g")
#conf.set("spark.core.connection.ack.wait.timeout", "1200")

# create the context
sc = pyspark.SparkContext(conf=conf)

# do something to prove it works
rdd = sc.parallelize(range(100000000))
rdd.sumApprox(3)

#textFile = sc.textFile("README.md")

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=Python Spark SQL basic example, master=local[*]) created by getOrCreate at <ipython-input-1-0c26c15c4eb5>:4 

In [2]:
#textFile = sc.textFile("README.md")
textFile = sc.textFile("/tmp/data/Kiosk-OCC_201709140800.csv")
#textFile = sc.textFile("s3://itk-redshift-staging/Kiosk-OCC_201709140800.csv")
textFile.count()

139

In [14]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("spark://spark-master:7077") \
    .appName("PemiSpark") \
    .config("spark.sql.warehouse.dir", "/tmp/data/spark-warehouse") \
    .getOrCreate()
    
spark#.getOrCreate()
spark.builder.getOrCreate()

In [7]:
sc = spark.sparkContext
sc.textFile("/tmp/data/Kiosk-OCC_201709140800.csv").count()

139

In [4]:
# spark is an existing SparkSession
df = spark.read.json("/tmp/data/people.json")
# Displays the content of the DataFrame to stdout
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [5]:
import pandas as pd

pd_df = pd.DataFrame({
    'id': [1,2,3],
    'name': ['one', 'two', 'three']
})

#spark_df = context.createDataFrame(pandas_df)
spark_df = spark.createDataFrame(pd_df)
spark_df.show()

spark_df.toPandas()

+---+-----+
| id| name|
+---+-----+
|  1|  one|
|  2|  two|
|  3|three|
+---+-----+



,id,name
0,1,one
1,2,two
2,3,three


In [1]:
import pemi
import pemi.data

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("spark://spark-master:7077") \
    .appName("PemiSpark") \
    .config("spark.sql.warehouse.dir", "/tmp/data/spark-warehouse") \
    .getOrCreate()
    

sales_schema = {
    'beer_id':  {'ftype': 'integer', 'required': True},
    'sold_at':  {'ftype': 'date', 'in_format': '%m/%d/%Y', 'required': True},
    'quantity': {'ftype': 'integer', 'required': True}
}

beers_schema = {
    'id':       {'ftype': 'integer', 'required': True},
    'name':     {'ftype': 'string', 'required': True},
    'style':    {'ftype': 'string'},
    'abv':      {'ftype': 'float'},
    'price':    {'ftype': 'decimal', 'precision': 16, 'scale': 2}
}
 
sales_table = pemi.data.Table(
    '''
    | beer_id | sold_at    | quantity |
    | -       | -          | -        |
    | 1       | 01/01/2017 | 3        |
    | 2       | 01/02/2017 | 3        |
    | 3       | 01/03/2017 | 5        |
    | 4       | 01/04/2017 | 8        |
    | 5       | 01/04/2017 | 6        |
    | 1       | 01/06/2017 | 1        |
    ''',
    schema=sales_schema,
    fake_with={
        'beer_id': { 'valid': lambda: pemi.data.fake.random_int(1,4) },
        'sold_at': { 'valid': lambda: pemi.data.fake.date_time_this_decade().date() },
        'quantity': {'valid': lambda: pemi.data.fake.random_int(1,100) },
        'bumpkin': { 'valid': lambda: pemi.data.fake.word(['bumpkin A', 'bumpkin B', 'bumpkin C']) }
    }
)


beers_table = pemi.data.Table(
    '''
    | id | name          | style |
    | -  | -             | -     |
    | 1  | SpinCyle      | IPA   |
    | 2  | OldStyle      | Pale  |
    | 3  | Pipewrench    | IPA   |
    | 4  | AbstRedRibbon | Lager |
    ''',
    schema=beers_schema,
    fake_with={
        'abv': {'valid': lambda: pemi.data.fake.pydecimal(2, 2, positive=True)},
        'price': {'valid': lambda: pemi.data.fake.pydecimal(2, 2, positive=True)}
    }
)

beers_df = spark.createDataFrame(beers_table.df)
sales_df = spark.createDataFrame(sales_table.df)
sales_df.show()
beers_df.show()


beers_df.createOrReplaceTempView('beers')
sales_df.createOrReplaceTempView('sales')

beer_sales_df = spark.sql('''
    SELECT
      sales.beer_id,
      beers.name,
      beers.style,
      sales.sold_at,
      sales.quantity,
      CAST(beers.price as decimal(10, 2)) AS unit_price,
      CAST(beers.price * sales.quantity AS DECIMAL(10,2)) AS sell_price
    FROM
      sales
    LEFT JOIN
      beers
    ON
      sales.beer_id = beers.id
''')

beer_sales_df.show()

+-------+----------+--------+
|beer_id|   sold_at|quantity|
+-------+----------+--------+
|      1|2017-01-01|       3|
|      2|2017-01-02|       3|
|      3|2017-01-03|       5|
|      4|2017-01-04|       8|
|      5|2017-01-04|       6|
|      1|2017-01-06|       1|
+-------+----------+--------+

+---+-------------+-----+--------------------+--------------------+
| id|         name|style|                 abv|               price|
+---+-------------+-----+--------------------+--------------------+
|  1|     SpinCyle|  IPA|51.82000000000000...|2.810000000000000000|
|  2|     OldStyle| Pale|22.23000000000000...|97.40000000000000...|
|  3|   Pipewrench|  IPA|60.29000000000000...|14.80000000000000...|
|  4|AbstRedRibbon|Lager|43.30000000000000...|85.38000000000000...|
+---+-------------+-----+--------------------+--------------------+

+-------+-------------+-----+----------+--------+----------+----------+
|beer_id|         name|style|   sold_at|quantity|unit_price|sell_price|
+-------+-

In [2]:
df = beer_sales_df.toPandas()
df

,beer_id,name,style,sold_at,quantity,unit_price,sell_price
0,5,None,None,2017-01-04,6,None,None
1,1,SpinCyle,IPA,2017-01-01,3,99.780000000000000000,299.340000000000000000
2,1,SpinCyle,IPA,2017-01-06,1,99.780000000000000000,99.780000000000000000
3,3,Pipewrench,IPA,2017-01-03,5,1.600000000000000000,8.000000000000000000
4,2,OldStyle,Pale,2017-01-02,3,5.200000000000000000,15.600000000000000000
5,4,AbstRedRibbon,Lager,2017-01-04,8,51.620000000000000000,412.960000000000000000


In [8]:
import importlib
importlib.reload(pemi.field)
importlib.reload(pemi)

schema=pemi.Schema({
    'beer_id':    {'ftype': 'integer', 'required': True},
    'name':       {'ftype': 'string'},
    'style':      {'ftype': 'string'},
    'sold_at':    {'ftype': 'date', 'in_format': '%m/%d/%Y', 'required': True},
    'quantity':   {'ftype': 'integer', 'required': True},
    'unit_price': {'ftype': 'decimal', 'precision': 16, 'scale': 2},
    'sell_price': {'ftype': 'decimal', 'precision': 16, 'scale': 2}
})


#df['sold_at'].apply(schema['sold_at'].in_converter)
df['unit_price'].apply(schema['unit_price'].in_converter)
#df['unit_price'].apply(lambda x: x.__class__.__name__)

0                     None
1    99.780000000000000000
2    99.780000000000000000
3     1.600000000000000000
4     5.200000000000000000
5    51.620000000000000000
Name: unit_price, dtype: object